# Installing required packages

In [ ]:
!pip install colabcode 

In [ ]:
!pip install ultralytics

In [ ]:
!pip install fastapi

# FastAPI Code

In [ ]:

import base64, io, uvicorn, cv2
from colabcode import ColabCode
from fastapi import FastAPI, Form, HTTPException,WebSocket
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import numpy as np
from YOLOv8 import YOLOv8

cc = ColabCode(port = 12000, code = False)
app = FastAPI()    

loaded_model = YOLOv8()

class ImageRequest(BaseModel):
  image_base64: str

# My endpoint
@app.get("/")
async def read_root():
  return {"hi": "this is blind-guide api"}

@app.post("/detect-image/", tags=["list"])
async def post_detect_image(image_request: ImageRequest):
  encoded_image = image_request.image_base64
  img_data = base64.b64decode(encoded_image)
  results = classify_image(img_data)
  return results

@app.websocket("/detect-image/")
async def websocket_endpoint(websocket: WebSocket):
    while True:
        encoded_image = await websocket.receive_text()
        img_data = base64.b64decode(encoded_image)
        results = classify_image(img_data)
        await websocket.send_json(results)

def classify_image(img_data):
    img = cv2.imdecode(np.fromstring(img_data, np.uint8), cv2.IMREAD_COLOR)
    conf = 0.7
    txt, image = loaded_model.predict(img, conf)
    retval, buffer = cv2.imencode(".jpg", image)
    encoded_processed_image = base64.b64encode(buffer)
    decoded_processed_image = base64.b64decode(encoded_processed_image)
    with open("decoded_processed_image.png", "wb") as fh:
      fh.write(decoded_processed_image)
    return {"list": txt}

# Start the server

In [ ]:
cc.run_app(app=app)